In [305]:
from sklearn.datasets import load_svmlight_file
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy
import pandas

In [235]:
data=load_svmlight_file("C:\\Users\\Miao\\Desktop\\Courses\\S675 Statistical Learning\\final\\corpus.dat")

In [6]:
len(data)

2

In [7]:
data[0]

<593x204562 sparse matrix of type '<type 'numpy.float64'>'
	with 6384862 stored elements in Compressed Sparse Row format>

In [9]:
len(data[1])

593

#### preprocess doc_word matrix (data[0]), remove words that occur in fewer than 10 docs

In [263]:
###this tries to prune the raw doc_word matrix by removing words occuring in fewere than 10 docs
###but I aborted this program because it's taking too long, after running for 6 hours; there are 204562 columns to loop through.
ncols=data[0].shape[1]  #num of columns, i.e. num of words
selected_cols=[]

for i in range(ncols):
    if len(data[0].getcol(i).nonzero()[0])>10:
        selected_cols.append(i)

selected_data=data[0][:, selected_cols]
selected_data.shape
###the pruned matrix will be the basis for calculating co-occurrence matrix, but since we aborted this step
#for co-occurrence, we just calcualte cooccurrence of specific words, instead of the whole co-occurrence matrix

KeyboardInterrupt: 

In [271]:
ncols=data[0].shape[1]  #num of columns, i.e. num of words
ncols
selected_data=data[0][:, range(2)]
selected_data.shape
len(data[0].getcol(2).nonzero()[0])
selected_cols.append(2)
selected_cols

[0,
 2,
 3,
 4,
 7,
 9,
 14,
 21,
 23,
 24,
 28,
 30,
 35,
 37,
 38,
 45,
 48,
 49,
 53,
 55,
 60,
 68,
 70,
 71,
 72,
 73,
 76,
 78,
 81,
 85,
 86,
 87,
 88,
 91,
 96,
 97,
 98,
 99,
 100,
 103,
 114,
 115,
 118,
 119,
 121,
 124,
 125,
 128,
 131,
 132,
 133,
 140,
 142,
 144,
 146,
 147,
 151,
 152,
 153,
 157,
 158,
 165,
 166,
 168,
 170,
 174,
 181,
 183,
 184,
 186,
 187,
 190,
 192,
 193,
 196,
 198,
 199,
 201,
 203,
 215,
 216,
 218,
 221,
 226,
 228,
 229,
 232,
 237,
 238,
 239,
 249,
 252,
 253,
 254,
 256,
 258,
 262,
 264,
 266,
 267,
 268,
 275,
 276,
 280,
 281,
 282,
 283,
 284,
 287,
 290,
 291,
 293,
 295,
 296,
 310,
 317,
 318,
 319,
 321,
 323,
 324,
 326,
 329,
 332,
 333,
 334,
 338,
 339,
 340,
 344,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 357,
 360,
 365,
 366,
 369,
 371,
 372,
 373,
 377,
 384,
 388,
 390,
 391,
 393,
 394,
 395,
 396,
 397,
 398,
 404,
 407,
 409,
 410,
 411,
 412,
 415,
 416,
 417,
 419,
 421,
 422,
 424,
 425,
 426,
 427,
 428,
 434,


In [41]:
#transform sparse matrix of word counts to sparse matrix of tf*idf
#norm=I2, use IDF, tf=tf+tf*idf 
#reference (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)
tfidfTransformer = TfidfTransformer(norm='l1')
data_tfidf = tfidfTransformer.fit_transform(data[0])

In [42]:
#using latent semantic analysis; when substract column means from column values, it's the same as PCA. 
#reference http://scikit-learn.org/stable/modules/decomposition.html
#we can't center the data_tfidf here, as it will destroy the sparsity (http://scikit-learn.org/stable/modules/preprocessing.html)
svd=TruncatedSVD(n_components=2)
x = svd.fit(data_tfidf)
explained_variance = svd.explained_variance_ratio_.sum()
print explained_variance

#made it a function in python/test.py 

#it turned out using raw count (data) matrix resulted in much higher variance

0.0260323009056


In [45]:
data_tfidf.shape
data_tfidf.getrow(0)

<1x204562 sparse matrix of type '<type 'numpy.float64'>'
	with 13021 stored elements in Compressed Sparse Row format>

In [57]:
%run python/test.py

In [58]:
testmethod()

hello world!


## Apply Truncated SVD, something similar to PCA to the doc_word matrix

In [59]:
def pcaOnSparseMatrix (n_components, sparseMatrix):
    svd=TruncatedSVD(n_components)
    X = svd.fit(sparseMatrix)
    explained_variance = svd.explained_variance_ratio_.sum()
    return explained_variance   

In [61]:
pcaOnSparseMatrix (20, data[0])

0.71932344647114388

In [86]:
#given a sparse matrix, centralize it by substrcting cell value from its column mean
def getCentralMatrix (sparseMatrix):
    for i in range(data_tfidf.shape[1]):
        col=data_tfidf.getcol(i)
        colmean=numpy.mean(col.toarray())
        
col=data_tfidf.getcol(0)
#data_tfidf.mean(data_tfidf[0])

In [99]:
#col-numpy.mean(col.toarray())
type(data_tfidf.sum(axis=0))
#numpy.mean(col)

numpy.matrixlib.defmatrix.matrix

In [108]:
word_doc_tdidf = csr_matrix.transpose(data_tfidf)

In [109]:
pcaOnSparseMatrix (20, word_doc_tdidf)

0.25119816686793589

In [111]:
pcaOnSparseMatrix (20, data_tfidf)

0.21976027895420441

In [339]:
#word_doc = csr_matrix.transpose(data[0])
#pcaOnSparseMatrix (20, word_doc)
svd=TruncatedSVD(n_components=10)
X = svd.fit_transform(data[0])
#get mapping from word to PC
#pandas.DataFrame(svd.components_, index=["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10","PC11","PC12","PC13","PC14","PC15","PC16","PC17","PC18","PC19","PC20"])

In [319]:
#get explained_variance for SVD
explained_variance = svd.explained_variance_ratio_.sum()
print explained_variance

0.719325237635


In [320]:
print pcaOnSparseMatrix (2, data[0])
print pcaOnSparseMatrix (40, data[0])
print pcaOnSparseMatrix (60, data[0])
print pcaOnSparseMatrix (80, data[0])

0.333150098171
0.81563366975
0.865783999522
0.896571856106


In [321]:
print pcaOnSparseMatrix (10, data[0])
print pcaOnSparseMatrix (30, data[0])
print pcaOnSparseMatrix (50, data[0])
print pcaOnSparseMatrix (70, data[0])
print pcaOnSparseMatrix (90, data[0])
print pcaOnSparseMatrix (100, data[0])

0.614157229237
0.775936808712
0.843744565519
0.882888210901
0.908383121804
0.918405932645


In [115]:
word_doc = csr_matrix.transpose(data[0])
svd=TruncatedSVD(n_components=10)
X = svd.fit_transform(word_doc)
X.shape

(204562L, 10L)

## Calculate word similarity, with word represented by PCs
and then we compare it with words represented by its co-occuring words

In [122]:
pc_word_matrix=pandas.DataFrame(svd.components_, index=["PC1","PC2"])
type(pc_word_matrix)

pandas.core.frame.DataFrame

In [129]:
w0_inPcs=[w[0] for w in svd.components_]
w1_inPcs=[w[1] for w in svd.components_]

In [137]:
#compute cosine similairty between 2 words (represented in PCs)
#we have it substracted by 1 because cosine() computes cosine distance, it actually returns 1-cossim()
#http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine
1-cosine(w0_inPcs, w1_inPcs)

0.99353392267172802

## now deal with reading into the dictionary

read it into a list
also build a hash (dictionary), so that given a word, such as 'history' or 'hlstory' we can find its index in the array through searching in the hash

In [138]:
wordlist=[]
f=open("C:\\Users\\Miao\\Desktop\\Courses\\S675 Statistical Learning\\final\\vocab.txt", 'r')
for line in f:
    wordlist.append(line.strip())
f.close()

In [141]:
wordlist

['a',
 'a_nd',
 'aa',
 'aaa',
 'aaaa',
 'aac',
 'aachen',
 'aad',
 'aadth',
 'aae',
 'aaer',
 'aaf',
 'aah',
 'aahmes',
 'aai',
 'aaid',
 'aaion',
 'aaions',
 'aaith',
 'aaj',
 'aak',
 'aal',
 'aalensis',
 'aam',
 'aan',
 'aana',
 'aand',
 'aans',
 'aao',
 'aap',
 'aar',
 'aarau',
 'aard',
 'aare',
 'aargau',
 'aaron',
 'aaronite',
 'aas',
 'aat',
 'aatat',
 'aatr',
 'aav',
 'aavaan',
 'aavay',
 'aax',
 'aay',
 'aaya',
 'aays',
 'ab',
 'aba',
 'ababde',
 'ababdeh',
 'ababil',
 'aback',
 'abacus',
 'abad',
 'abada',
 'abade',
 'abades',
 'abadia',
 'abaft',
 'abagi',
 'abai',
 'abaixo',
 'abaj',
 'abajo',
 'abal',
 'abalorios',
 'aban',
 'abancay',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abandonna',
 'abandonnant',
 'abandonne',
 'abandonnee',
 'abandonner',
 'abandonnes',
 'abandono',
 'abandons',
 'abang',
 'abarbenel',
 'abaris',
 'abas',
 'abase',
 'abased',
 'abasement',
 'abases',
 'abash',
 'abashed',
 'abasing',
 'abasis',
 'abassines',
 'abat',
 'abate',
 'ab

In [335]:
word_index_hash=dict(zip(wordlist, range(len(wordlist))))
print word_index_hash['hlstory']
print word_index_hash['himself']

86261
85722


In [149]:
#define a function here: calculating cosine similarity between two words represented by PCs
def word_similarity_basedOnPCs(w1, w2):
    idx1=word_index_hash[w1]
    idx2=word_index_hash[w2]
    
    w1_inPCs=[w[idx1] for w in svd.components_]
    w2_inPCs=[w[idx2] for w in svd.components_]
    
    return 1-cosine(w1_inPCs, w2_inPCs)

In [322]:
word_similarity_basedOnPCs('hlstory', 'historv')

0.3346477965155471

In [340]:
#this is for 10 PCs
print word_similarity_basedOnPCs('hlstory', 'historv')
print word_similarity_basedOnPCs('himselt', 'himself')
print word_similarity_basedOnPCs('historv', 'riage')
print word_similarity_basedOnPCs('himselt', 'riage')

0.868327693758
0.607230627983
0.521518637142
-0.647791492866


In [336]:
#this is for 20 PCS
print word_similarity_basedOnPCs('hlstory', 'historv')
print word_similarity_basedOnPCs('himselt', 'himself')
print word_similarity_basedOnPCs('historv', 'riage')
print word_similarity_basedOnPCs('himselt', 'riage')

0.334647796516
-0.113503735498
-0.202564632201
-0.0425061429269


In [338]:
#this is for 60 PCs
print word_similarity_basedOnPCs('hlstory', 'historv')
print word_similarity_basedOnPCs('himselt', 'himself')
print word_similarity_basedOnPCs('historv', 'riage')
print word_similarity_basedOnPCs('himselt', 'riage')

-0.279018795692
0.0551483751981
-0.314858738887
-0.0636578833615


In [327]:
#get vector of PCS given a word
idx=word_index_hash['historv']
w_inPCs=[w[idx] for w in svd.components_]
w_inPCs

[0.00015217275155651251,
 0.00013434925512479854,
 -0.00018419881671763959,
 -6.9062195680203144e-05,
 -0.00037400490179892231,
 -0.00025386447564968042,
 -0.00015974273709882874,
 0.00057556578242177157,
 0.00015906150819033595,
 -0.00019404735766652696,
 -0.00011119511109342336,
 -4.3633642549517927e-05,
 0.00014540994302358345,
 -0.00024142896878784145,
 -3.6505998134993491e-05,
 -0.0002686467136335702,
 -0.00019919261089652965,
 0.0004944824663092838,
 -2.933356770681123e-05,
 0.00010887418285739097]

In [328]:
#get vector of documents for a word
idx=word_index_hash['historv']
data[0].getcol(idx).toarray()

array([[  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  1.],
       [  0.],
       [  0.],
       [  2.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  

## here we calculate word co-occurrence matrix from doc-word sparse matrix

In [258]:
from scipy.sparse import coo_matrix
row  = numpy.array([0, 3, 1, 0])
col  = numpy.array([0, 3, 1, 2])
datatest = numpy.array([4, 5, 7, 9])
toy=coo_matrix((datatest, (row, col)), shape=(4, 4)).toarray()
print toy
print toy[:, range(2)]

[[4 0 9 0]
 [0 7 0 0]
 [0 0 0 0]
 [0 0 0 5]]
[[4 0]
 [0 7]
 [0 0]
 [0 0]]


In [160]:
toy.nonzero()

(array([0, 0, 1, 3], dtype=int64), array([0, 2, 1, 3], dtype=int64))

In [176]:
testcoo=coo_matrix(data_tfidf)

In [183]:
testcoo.getcol(0).nonzero()

(array([  0,   3,   4,   5,   9,  13,  15,  16,  22,  33,  34,  39,  41,
         67,  68,  71,  72,  75,  80,  81,  89,  95,  97, 109, 112, 115,
        122, 128, 135, 136, 151, 155, 162, 168, 177, 186, 187, 188, 191,
        202, 206, 207, 209, 210, 211, 214, 216, 239, 260, 284, 322, 323,
        341, 361, 375, 380, 382, 388, 395, 399, 403, 412, 425, 431, 449,
        459, 464, 472, 490, 500, 515, 532, 553, 558, 566, 589, 590, 592]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [187]:
testcoo.getcol(0).nonzero()[0][1]

3

In [209]:
col0=data_tfidf.getcol(0).nonzero()
col0

(array([  0,   3,   4,   5,   9,  13,  15,  16,  22,  33,  34,  39,  41,
         67,  68,  71,  72,  75,  80,  81,  89,  95,  97, 109, 112, 115,
        122, 128, 135, 136, 151, 155, 162, 168, 177, 186, 187, 188, 191,
        202, 206, 207, 209, 210, 211, 214, 216, 239, 260, 284, 322, 323,
        341, 361, 375, 380, 382, 388, 395, 399, 403, 412, 425, 431, 449,
        459, 464, 472, 490, 500, 515, 532, 553, 558, 566, 589, 590, 592]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [210]:
col1=data_tfidf.getcol(1).nonzero()
col1

(array([177, 188, 202, 216, 491]), array([0, 0, 0, 0, 0]))

In [236]:
type(data_tfidf.getcol(1).nonzero()[0])
data_tfidf.shape[1]
data[0].shape

(593, 204562)

In [211]:
overlap=numpy.intersect1d(col0[0], col1[0])
overlap

array([177, 188, 202, 216])

In [238]:
sum=0

for k in overlap:
    elem1=data_tfidf.getcol(0)[k].toarray()[0][0]
    print elem1
    
    elem2=data_tfidf.getcol(1)[k].toarray()[0][0]
    print elem2
    
    elems=numpy.array([elem1, elem2])
    print elems
    print "min: "+str(numpy.amin(elems))
    print type(numpy.amin(elems))
    
    sum+=numpy.amin(elems)
    #print "min: "+str(min(elem1,elem2))
    
print sum
print type(sum)

5.771199723e-05
4.2805352146e-06
[  5.77119972e-05   4.28053521e-06]
min: 4.2805352146e-06
<type 'numpy.float64'>
0.000417483571079
6.19300392742e-05
[  4.17483571e-04   6.19300393e-05]
min: 6.19300392742e-05
<type 'numpy.float64'>
1.27087418086e-05
1.17826923898e-05
[  1.27087418e-05   1.17826924e-05]
min: 1.17826923898e-05
<type 'numpy.float64'>
6.44963213906e-05
2.9898330089e-05
[  6.44963214e-05   2.98983301e-05]
min: 2.9898330089e-05
<type 'numpy.float64'>
0.000107891596968
<type 'numpy.float64'>


In [214]:
data_tfidf.getcol(0)[177].toarray()

array([[  5.77119972e-05]])

In [250]:
#then make a SparseMatrix according to http://www.foldl.me/2014/glove-python/
#based on the above scripts, build up the co-occurrence matrix form data matrix
#input a word_doc matrix, output the cocurrence matrix
#if in a doc, word1 occurs 4 times, word2 occurs 6 times, then I count their co-occurrence as min(4,6)=4
def build_cooccur_matrix (word_doc_matrix):
    
    n_words=word_doc_matrix.shape[1] #number of words
    cooccurrence_matrix=lil_matrix((n_words, n_words), dtype=numpy.float64)
    
    #get column pairs from the matrix, we do it by column because SparseMatrix uses Column storage
    for c1 in range(n_words): #column2
        for c2 in range(n_words):  #for column2
            col1=data_tfidf.getcol(c1).nonzero()
            col2=data_tfidf.getcol(c2).nonzero()
            #get overlap elements in the two columns, we need them for co-occurrence; columns are words.
            #e.g. if both column 1 and column 2 has non-zero values for row k, then we count it as a co-occurrence
            #cooccrrence(word_1, word2)=sum of min(data[1,k], data[2,k]) for the overlapping elements
            overlap=numpy.intersect1d(col1[0], col2[0]) #overlap elements of column 1 and 2
            
            coocurr_value=0 #for recording cooccurrence value of word_1 and word_2
            
            for k in overlap:
                elem1=word_doc_matrix.getcol(c1)[k].toarray()[0][0]
                elem2=word_doc_matrix.getcol(c2)[k].toarray()[0][0]
                
                elems=numpy.array([elem1, elem2])
                coocurr_value+=numpy.amin(elems)
                
            cooccurrence_matrix[c1, c2]=coocurr_value
            cooccurrence_matrix[c2, c1]=coocurr_value
                

    return cooccurrence_matrix

In [251]:
cooccurrence_matrix=build_cooccur_matrix(data[0])
###it's taking too long, so aborted the program, it will be a 204562*204562 matrix...

KeyboardInterrupt: 

### Here we obtain doc similiarity based on word vectors from word_doc matrix

In [294]:
idx1=word_index_hash['hlstory']
idx2=word_index_hash['historv']

docvec1=data[0].getcol(idx1).toarray()
docvec2=data[0].getcol(idx2).toarray()

1-cosine(docvec1, docvec2)


0.028306925853614828

In [295]:
#given 2 words, calculate their similarity based on in what document they occur
#a word's vector here is all the documents it has occurred, can be obtained from word_doc matrix
def wordSimilarityBasedOnDocVec(word_doc_matrix, w1, w2):
    idx1=word_index_hash[w1]
    idx2=word_index_hash[w2]
    
    docvec1=word_doc_matrix.getcol(idx1).toarray()
    docvec2=word_doc_matrix.getcol(idx2).toarray()
    
    return 1-cosine(docvec1, docvec2)
    

In [297]:
wordSimilarityBasedOnDocVec(data[0], 'hlstory', 'historv')

0.028306925853614828

In [343]:
print wordSimilarityBasedOnDocVec(data[0], 'hlstory', 'historv')
print wordSimilarityBasedOnDocVec(data[0], 'himselt', 'himself')
print wordSimilarityBasedOnDocVec(data[0], 'historv', 'riage')
print wordSimilarityBasedOnDocVec(data[0], 'himselt', 'riage')

0.0283069258536
0.0
0.078072600677
0.0267049381997


### K-means clustering on document-word matrix

In [302]:
km = KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1, verbose=True)

In [303]:
km.fit(data[0])

Initialization complete
Iteration  0, inertia 1976681818.000
Iteration  1, inertia 1659199417.779
Iteration  2, inertia 1637356293.531
Iteration  3, inertia 1622846354.323
Iteration  4, inertia 1620377879.539
Iteration  5, inertia 1619919276.548
Iteration  6, inertia 1619091789.334
Converged at iteration 6


KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=True)

In [307]:
#The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample. 
#The Silhouette Coefficient for a sample is (b - a) / max(a, b).
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(data[0], km.labels_, sample_size=1000))

Silhouette Coefficient: 0.383


In [347]:
#get Silhouette coefficient for clustering result
def clusterAndGetSilhouetteCoef(data, n_clusters):
    km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1, verbose=True)
    km.fit(data)
    return metrics.silhouette_score(data, km.labels_, sample_size=1000)

In [348]:
clusterAndGetSilhouetteCoef(data[0], 10)

Initialization complete
Iteration  0, inertia 2406258056.000
Iteration  1, inertia 2101185834.829
Iteration  2, inertia 2075810706.425
Iteration  3, inertia 2074197054.718
Converged at iteration 3


0.5402546431251406

In [346]:
clusterAndGetSilhouetteCoef(data[0], 20)

0.51279720911233428

In [349]:
clusterAndGetSilhouetteCoef(data[0], 30)

Initialization complete
Iteration  0, inertia 1546426139.000
Iteration  1, inertia 1354477239.732
Iteration  2, inertia 1337592291.058
Iteration  3, inertia 1331814707.011
Iteration  4, inertia 1330332759.597
Iteration  5, inertia 1329313686.954
Iteration  6, inertia 1328518836.208
Iteration  7, inertia 1326682477.368
Iteration  8, inertia 1325773063.771
Iteration  9, inertia 1325468017.089
Iteration 10, inertia 1325335156.650
Converged at iteration 10


0.40445106004206155

In [350]:
clusterAndGetSilhouetteCoef(data[0], 40)

Initialization complete
Iteration  0, inertia 1361518178.000
Iteration  1, inertia 1137677020.244
Iteration  2, inertia 1132849352.343
Iteration  3, inertia 1132213222.104
Iteration  4, inertia 1131895837.617
Iteration  5, inertia 1131676570.271
Iteration  6, inertia 1131287160.464
Iteration  7, inertia 1131191866.664
Iteration  8, inertia 1131056782.105
Iteration  9, inertia 1130978265.743
Iteration 10, inertia 1130927283.870
Iteration 11, inertia 1130917454.399
Iteration 12, inertia 1130907031.260
Converged at iteration 12


0.31384905403882807

In [351]:
clusterAndGetSilhouetteCoef(data[0], 50)

Initialization complete
Iteration  0, inertia 1141417253.000
Iteration  1, inertia 980731217.669
Iteration  2, inertia 968114113.748
Iteration  3, inertia 965543659.458
Iteration  4, inertia 964273837.305
Iteration  5, inertia 963687455.162
Iteration  6, inertia 963331209.030
Iteration  7, inertia 963090235.364
Iteration  8, inertia 963062570.900
Iteration  9, inertia 963047181.214
Converged at iteration 9


0.20205560562056302

In [352]:
clusterAndGetSilhouetteCoef(data[0], 60)

Initialization complete
Iteration  0, inertia 947417017.000
Iteration  1, inertia 808947545.414
Iteration  2, inertia 800440967.529
Iteration  3, inertia 797283170.537
Iteration  4, inertia 796356260.842
Iteration  5, inertia 795859859.282
Iteration  6, inertia 795827544.279
Converged at iteration 6


0.2376479235768805

In [353]:
clusterAndGetSilhouetteCoef(data[0], 70)

Initialization complete
Iteration  0, inertia 832628555.000
Iteration  1, inertia 724360598.469
Iteration  2, inertia 721435682.275
Iteration  3, inertia 720989138.145
Iteration  4, inertia 720807103.989
Iteration  5, inertia 720757145.191
Iteration  6, inertia 720639065.315
Iteration  7, inertia 720470453.841
Iteration  8, inertia 720260263.794
Iteration  9, inertia 720124624.594
Iteration 10, inertia 720055755.536
Iteration 11, inertia 719979689.899
Iteration 12, inertia 719932347.294
Iteration 13, inertia 719906567.070
Converged at iteration 13


0.29433449920150867

In [354]:
clusterAndGetSilhouetteCoef(data[0], 80)

Initialization complete
Iteration  0, inertia 775156687.000
Iteration  1, inertia 659042679.980
Iteration  2, inertia 653541301.157
Iteration  3, inertia 650857129.815
Iteration  4, inertia 648534500.152
Iteration  5, inertia 647583424.446
Iteration  6, inertia 646560412.673
Iteration  7, inertia 646010935.863
Iteration  8, inertia 645764041.820
Iteration  9, inertia 645526711.843
Converged at iteration 9


0.28041505608648298

In [355]:
clusterAndGetSilhouetteCoef(data[0], 90)

Initialization complete
Iteration  0, inertia 685373957.000
Iteration  1, inertia 574222683.964
Iteration  2, inertia 569952328.178
Iteration  3, inertia 565919318.445
Iteration  4, inertia 563780154.580
Iteration  5, inertia 562720409.652
Iteration  6, inertia 562332256.911
Iteration  7, inertia 562142877.081
Converged at iteration 7


0.27975255923038717

In [356]:
clusterAndGetSilhouetteCoef(data[0], 100)

Initialization complete
Iteration  0, inertia 649516530.000
Iteration  1, inertia 532540042.201
Iteration  2, inertia 527733582.321
Iteration  3, inertia 526038370.499
Iteration  4, inertia 524576045.795
Iteration  5, inertia 523109113.399
Iteration  6, inertia 522740728.256
Iteration  7, inertia 522648306.857
Iteration  8, inertia 522576551.891
Iteration  9, inertia 522408678.989
Iteration 10, inertia 522369000.972
Converged at iteration 10


0.28598567057044871

In [310]:
X.shape

(593L, 2L)

In [311]:
km_onPC = KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1, verbose=True)

In [312]:
km_onPC.fit(X)

Initialization complete
Iteration  0, inertia 13139122.916
Iteration  1, inertia 10852053.894
Iteration  2, inertia 10499448.498
Iteration  3, inertia 10389777.925
Iteration  4, inertia 10256363.453
Iteration  5, inertia 10031508.548
Iteration  6, inertia 9864388.466
Iteration  7, inertia 9761203.441
Iteration  8, inertia 9701866.007
Iteration  9, inertia 9662486.596
Iteration 10, inertia 9627938.711
Iteration 11, inertia 9598416.772
Iteration 12, inertia 9573832.586
Converged at iteration 12


KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=True)

In [313]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(data[0], km_onPC.labels_, sample_size=1000))

Silhouette Coefficient: -0.116


In [314]:
data[0].shape

(593, 204562)

In [316]:
idx2=word_index_hash['historv']
idx2

86129